# Further Analysis

## This notebook is for interpretting my model for production

First I import my dependencies in order to manipulate data, performing calculations, getting word counts and frequencies, and machine learning classification testing and modeling.

In [39]:
# Manipulates data
import pandas as pd

# Performs numerical computations
import numpy as np

# Machine Learning things 
# The vectorizers are for performing analysis on word counts/frequencies
from sklearn.feature_extraction.text import CountVectorizer

# These are for classification and statistical analysis
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

Next, I will import my JSON and prepare it to be stemmed and vectorized.

In [40]:
sm = pd.read_json("../data_fetching/json2/SequelMemes.json")
pm = pd.read_json("../data_fetching/json2/PrequelMemes.json")
sm_titles = sm[["title"]]
pm_titles = pm[["title"]]

sm_titles["is_sequel_meme"] = 1
pm_titles["is_sequel_meme"] = 0

meme_titles = pd.concat([pm_titles,sm_titles])
corpus = [title[0] for title in meme_titles[["title"]].values]

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Then I will vectorize the titles and fit them to a Bernoulli Naive Bayes Classification Model

In [41]:
cvec = CountVectorizer(ngram_range=(1,3),max_df=.25, stop_words='english')
cvec.fit(corpus)
# Transform the corpus
X  = cvec.transform(corpus)
y = meme_titles[["is_sequel_meme"]]
X_train, X_test, y_train, y_test = train_test_split(X, y)

bnb = BernoulliNB()
bnb.fit(X_train, y_train)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

Finally, I will create a confusion matrix along with measuring the accuracy score

In [42]:
predictions = bnb.predict(X_test)

# https://stackoverflow.com/a/48412991/2548452
acc_score = accuracy_score(y_test, predictions)
conf_mat = confusion_matrix(
        y_test, predictions)

print(acc_score)
print(conf_mat)

0.7657618842452966
[[2558  763]
 [ 868 2774]]


Generally speaking, this model is 77% accurate with higher disposition of misclassification towards false positives rather than false negatives. This means that it generally mistakes posts titles from r/SequelMemes as if they were posts titles from r/PrequelMemes.

I think this makes sense because r/PrequelMemes is an older subreddit and has a larger number of posts. It may also be that there is shared language that may increase this misclassification rate given that both of these subreddits are very referenced based.

I also think that these subreddits have a very similar sense of humor and that gets encoded in the titles albeit with different sets of references.